### Input data

1. Download `Mutation_perturbation_model.csv` (training set) from this [repo](https://github.com/jishnu-lab/SWING/tree/main/Data/MutInt_Model). 
2. Download all Uniprot sequences and their identifiers from this [link](https://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/complete/uniprot_sprot.fasta.gz). 

In [ ]:
import pandas as pd
from Bio import SeqIO
import re

In [ ]:
record_dict = SeqIO.to_dict(SeqIO.parse("../uniprot_sprot.fasta", "fasta"))
kv = list(record_dict.items())
record_dict.clear()
for k, v in kv :
    new_k = (k.split('|')[1])
    record_dict[new_k] = str(v.seq)

In [ ]:
df = pd.read_csv('Mutation_perturbation_model.csv')

In [ ]:
mut_pattern = r'([A-Z])(\d+)([A-Z])'

wt_seq1s = []
wt_seq2s = []
mut_seq1s = []
mut_seq2s = []
targets = []

for i, row in df.iterrows():

    id1 = row['Mutation UPID']
    id2 = row['Interactor UPID']
    target = row['Y2H_score']

    mut = row['Mutation']
    
    seq1 = record_dict[id1]
    seq2 = record_dict[id2]

    mut_match = re.match(mut_pattern, mut)
    wt_res = mut_match.group(1) 
    res_num = int(mut_match.group(2))
    mut_res = mut_match.group(3)

    if wt_res != seq1[res_num-1]:
        print(f'Failed on index {i}')
        continue
    else:
        seq1_l = list(seq1)
        seq1_l[res_num-1] = mut_res
        mut_seq1 = ''.join(seq1_l)

    if seq1 == mut_seq1:
        print(f'Skipping on index {i}')
        continue

    wt_seq1s.append(seq1)
    wt_seq2s.append(seq2) 
    mut_seq1s.append(mut_seq1)
    mut_seq2s.append(seq2)
    targets.append(target)

In [ ]:
mut_df = pd.DataFrame({'seq1': wt_seq1s, 
                       'seq2': wt_seq2s, 
                       'seq1_mut': mut_seq1s,
                       'seq2_mut': mut_seq2s,
                       'target': targets})

In [ ]:
mut_df.to_csv('processed_data.csv')